In [1]:
using DataFrames
using Convex

In [5]:
# load data
listings = readtable("listings_cleaned_smaller.csv");

n = size(listings)[1]

# Code borrowed from http://blog.yhat.com/posts/julia-neural-networks.html
# Generate train/test split
is_train = shuffle(1:n .> floor(n * .25))
listings_train, listings_test = listings[is_train, :], listings[!is_train, :]
listings = listings_train;

In [6]:
# fit a linear model
x = copy(listings)
x[:Offset] = 1
delete!(x, :price)

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,amenities,square_feet,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,property_type_apartment,property_type_bed_breakfast,property_type_boat,property_type_bungalow,property_type_cabin,property_type_camper_rv,property_type_castle,property_type_cave,property_type_chalet,property_type_condominium,property_type_dorm,property_type_earth_house,property_type_entire_floor,property_type_guesthouse,property_type_house,property_type_hut,property_type_lighthouse,property_type_loft,property_type_other,property_type_parking_space,property_type_plane,property_type_tent,property_type_tipi,property_type_townhouse,property_type_train,property_type_treehouse,property_type_villa,property_type_yurt,room_type_entire_home_apt,room_type_private_room,room_type_shared_room,bed_type_airbed,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_no_refunds,cancellation_policy_strict,cancellation_policy_super_strict_60,Offset
1,49,0.0,0.0,0,1,6,1,1,34.04206443,-118.2532204,1,3,1.0,1,1,18,0,0,15,1,0,2,2,0,0,0,0,2,100,10,10,10,10,10,10,0,0.12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1
2,35,1.0,1.0,1,1,4,1,1,34.09483054,-118.2644041,1,3,1.0,2,2,10,0,0,80,1,0,3,1125,0,0,15,283,38,99,10,9,10,10,10,10,0,1.27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1
3,52,0.87,0.93,0,1,4,1,1,34.11247523,-118.3364378,1,2,1.0,1,1,12,0,0,20,2,20,1,5,22,52,80,355,41,94,10,9,10,10,10,9,0,1.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1
4,29,1.0,1.0,0,1,6,1,1,34.06455122,-118.4951368,0,3,1.0,1,2,14,0,0,0,1,0,1,1125,29,59,89,364,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
5,31,0.75,1.0,0,1,3,1,0,33.98847795,-118.4411853,1,2,1.0,1,1,23,0,0,0,1,0,1,1125,30,60,90,365,0,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
6,6,0.0,1.0,0,1,1,1,0,33.92738545,-118.2697351,1,2,2.0,1,2,13,0,100,20,1,0,5,1125,28,58,88,88,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
7,3,0.0,0.0,0,1,2,1,0,34.06783664,-118.3492394,1,2,1.5,1,1,17,0,0,0,1,0,1,1125,29,29,29,29,0,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
8,26,0.0,0.0,0,1,5,1,1,34.04959846,-118.4633752,1,1,1.0,1,1,17,0,445,0,1,0,1,1125,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
9,17,1.0,0.9,0,1,6,1,1,34.08844529,-118.3551676,1,3,1.0,1,1,17,0,0,0,1,15,1,1125,1,3,33,308,21,88,9,9,9,9,9,9,0,1.74,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
10,3,1.0,0.0,0,1,2,1,0,33.85585933,-118.124111,1,2,1.0,1,1,10,0,0,0,1,0,1,25,30,60,90,365,0,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1


In [7]:
X = convert(Array, x)
Y = convert(Array, listings[:price])
@show w = X\Y

30×79 Array{Real,2}:
 49  0.0   0.0   0  1  6  1  1  …  0  0  0  0  0  1  0  1  0  0  0  1
 35  1.0   1.0   1  1  4  1  1     0  0  0  0  0  1  0  0  0  1  0  1
 52  0.87  0.93  0  1  4  1  1     0  0  0  0  0  1  0  1  0  0  0  1
 29  1.0   1.0   0  1  6  1  1     0  0  0  0  0  1  1  0  0  0  0  1
 31  0.75  1.0   0  1  3  1  0     0  0  0  0  0  1  1  0  0  0  0  1
  6  0.0   1.0   0  1  1  1  0  …  0  0  0  0  0  1  1  0  0  0  0  1
  3  0.0   0.0   0  1  2  1  0     0  0  0  0  0  1  1  0  0  0  0  1
 26  0.0   0.0   0  1  5  1  1     0  0  0  0  0  1  1  0  0  0  0  1
 17  1.0   0.9   0  1  6  1  1     0  0  0  0  0  1  1  0  0  0  0  1
  3  1.0   0.0   0  1  2  1  0     0  0  0  0  0  1  1  0  0  0  0  1
 46  0.86  1.0   0  1  4  1  1  …  0  0  0  0  0  1  1  0  0  0  0  1
 71  1.0   0.0   0  1  3  1  0     0  0  0  0  0  1  0  0  0  1  0  1
 27  0.0   0.0   0  1  2  1  0     0  0  0  0  0  1  0  0  0  1  0  1
  ⋮                    ⋮        ⋱           ⋮              ⋮         

In [ ]:
f(x) = sum(w.*x)

x1 = X[1,:]
x2 = X[2,:]
x3 = X[3,:]
@show f(x1)
@show f(x2)
@show f(x3)

In [ ]:
function lasso(X,y; λ=1)
    @show d = size(X,2)
    @show w = Variable(d)
    @show p = minimize(sumsquares(X*w - y) + λ*norm(w,1))
    solve!(p)
    return w.value
end

# lasso(X, Y)

# Generate random problem data
m = 4;  n = 5
A = randn(m, n); b = randn(m, 1)

# Create a (column vector) variable of size n x 1.
x = Variable(n)

# The problem is to minimize ||Ax - b||^2 subject to x >= 0
# This can be done by: minimize(objective, constraints)
problem = minimize(sumsquares(A * x - b), [x >= 0])

# Solve the problem by calling solve!
solve!(problem)

# Check the status of the problem
problem.status # :Optimal, :Infeasible, :Unbounded etc.

# Get the optimal value
problem.optval